In [1]:
from utils import (
    load_untrained_model,
    count_nonzero_parameters
)

In [11]:
dict_ = load_untrained_model("DenseNet121")
model = dict_["model"]

In [2]:
'''DenseNet in PyTorch.'''
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

######## Bottleneck Classes ########
class Bottleneck(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(Bottleneck, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, 4*growth_rate, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(4*growth_rate)
        self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat([out,x], 1)
        return out
    
class BottleneckDepthWise(nn.Module):
    def __init__(self, in_planes, growth_rate):
        super(BottleneckDepthWise, self).__init__()
        inter_planes = 4 * growth_rate

        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, bias=False)

        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.dwconv = nn.Conv2d(inter_planes, inter_planes, kernel_size=3, padding=1, groups=inter_planes, bias=False)  # depthwise
        self.pwconv = nn.Conv2d(inter_planes, growth_rate, kernel_size=1, bias=False)  # pointwise

    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))             # 1x1 conv
        out = self.dwconv(F.relu(self.bn2(out)))          # depthwise 3x3
        out = self.pwconv(out)                            # pointwise 1x1
        out = torch.cat([out, x], 1)
        return out

######## Transition ########
class Transition(nn.Module):
    def __init__(self, in_planes, out_planes):
        super(Transition, self).__init__()
        self.bn = nn.BatchNorm2d(in_planes)
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.conv(F.relu(self.bn(x)))
        out = F.avg_pool2d(out, 2)
        return out

######## DenseNet ########
class DenseNet(nn.Module):
    def __init__(self, block, nblocks, growth_rate=12, reduction=0.5, num_classes=10):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate

        num_planes = 2*growth_rate
        self.conv1 = nn.Conv2d(3, num_planes, kernel_size=3, padding=1, bias=False)

        self.dense1 = self._make_dense_layers(block, num_planes, nblocks[0])
        num_planes += nblocks[0]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans1 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense2 = self._make_dense_layers(block, num_planes, nblocks[1])
        num_planes += nblocks[1]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans2 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense3 = self._make_dense_layers(block, num_planes, nblocks[2])
        num_planes += nblocks[2]*growth_rate
        out_planes = int(math.floor(num_planes*reduction))
        self.trans3 = Transition(num_planes, out_planes)
        num_planes = out_planes

        self.dense4 = self._make_dense_layers(block, num_planes, nblocks[3])
        num_planes += nblocks[3]*growth_rate

        self.bn = nn.BatchNorm2d(num_planes)
        self.linear = nn.Linear(num_planes, num_classes)

    def _make_dense_layers(self, block, in_planes, nblock):
        layers = []
        for i in range(nblock):
            layers.append(block(in_planes, self.growth_rate))
            in_planes += self.growth_rate
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.trans1(self.dense1(out))
        out = self.trans2(self.dense2(out))
        out = self.trans3(self.dense3(out))
        out = self.dense4(out)
        out = F.avg_pool2d(F.relu(self.bn(out)), 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [3]:
model_original = DenseNet(Bottleneck, [6,12,24,16], growth_rate=32)
model_depthwise = DenseNet(BottleneckDepthWise, [6,12,24,16], growth_rate=32)

In [4]:
print(f"Original DenseNet121 (Bottleneck): {count_nonzero_parameters(model_original):,} parameters")
print(f"Modified DenseNet121 (Depthwise): {count_nonzero_parameters(model_depthwise):,} parameters")

Original DenseNet121 (Bottleneck): 6,914,538 parameters
Modified DenseNet121 (Depthwise): 5,080,809 parameters
